#Image labeller with persistence (example)

This notebook demonstrates using the image labelling tool as an IPython Notebook plugin.

The kernel-side widget model is defined in the `ImageLabellingTool` class in the `labelling_tool` module.

In [ ]:
from IPython.display import display

import labelling_tool, labelling_tool_ipython

###Client side widget

Here is the client side widget implementation:

In [ ]:
%%javascript

require(["widgets/js/widget",
         "widgets/js/manager",
         "http://d3js.org/d3.v3.min.js",
         "static/labelling_tool.js"], function(widget, manager){
    /*
    Labeling tool view; links to the server side data structures
     */
    var ImageLabellingToolView = widget.DOMWidgetView.extend({
        render: function() {
            var self = this;

            // Register a custom IPython widget message handler for receiving messages from the Kernel
            this.model.on('msg:custom', this._on_custom_msg, this);


            // Get label classes, tool dimensions, and image ID set and initial image ID from the kernel
            var label_classes = self.model.get("label_classes");
            var tool_width = self.model.get("tool_width_");
            var tool_height = self.model.get("tool_height_");
            var image_ids = self.model.get('image_ids_');
            var initial_image_id = self.model.get('current_image_id_');


            // Callback function to allow the labelling tool to request an image
            var request_image_descriptor = function(image_id_str) {
                // Send a 'request_image_descriptor' message to the kernel requesting the
                // image identified by `image_id_str`
                self.send({msg_type: 'request_image_descriptor', image_id: image_id_str});
            };

            // Callback function to allow the labelling tool to send modified label data to the kernel
            var send_label_header = function(label_header) {
                // Send a 'label_header' message to the kernel, along with modified label data
                self.send({msg_type: 'label_header', label_header: label_header});
            };

            // Create the labelling tool
            self._labeling_tool = LabellingTool();
            // Initialise it, placing it into the widget element (`this.$el`).
            // Also give it the label classes, tool dimensions, image ID set, initial image ID and the callbacks above
            self._labeling_tool.initialise(this.$el, label_classes, tool_width, tool_height,
                                            image_ids, initial_image_id, request_image_descriptor, send_label_header);
        },


        _on_custom_msg: function(msg) {
            // Received a custom message from the kernel
            if (msg.msg_type === "set_image") {
                // 'set_image' message
                var in_image = msg.image;
                // Build the image descriptor to give to the labelling tool
                var image = {
                    width: in_image.width,
                    height: in_image.height,
                    href: in_image.href,
                    label_header: in_image.label_header
                };
                // Make the labelling tool switch images
                this._labeling_tool.setImage(image);
            }
        }
    });

    // Register the ImageLabelingToolView with the IPython widget manager.
    manager.WidgetManager.register_widget_view('ImageLabellingToolView', ImageLabellingToolView);
    console.log("registered");
});


##Example tool

In [ ]:
# Specify our 3 label classes.
# `LabelClass` parameters are: symbolic name, human readable name for UI, and RGB colour as list
label_classes = [labelling_tool.LabelClass('tree', 'Trees', [0, 255, 192]),
                 labelling_tool.LabelClass('building', 'Buldings', [255, 128, 0]),
                 labelling_tool.LabelClass('lake', 'Lake', [0, 128, 255]),
                 ]

# Define the tool dimensions
TOOL_WIDTH, TOOL_HEIGHT = 980, 480

# Load in .JPG images from the 'images' directory.
labelled_images = labelling_tool.PersistentLabelledImage.for_directory('images', image_filename_pattern='*.jpg')
print 'Loaded {0} images'.format(len(labelled_images))

# Create the labelling tool IPython widget and display it
labeller = labelling_tool_ipython.ImageLabellingTool(labelled_images=labelled_images, label_classes=label_classes,
                                                     tool_width=TOOL_WIDTH, tool_height=TOOL_HEIGHT)

display(labeller)

###Instructions for use

To navigate between images:

- Using the left and right arrows to navigate the images one by one
- Enter a number in the box at the top to navigate to a specific image

To label regions of the image:

- **Drawing regions onto the image:**
  - Click the *Draw poly* button
  - Within the image pane, left-click to draw polygonal corners of your region
  - When you have finished the region, right-click to stop
  - You are still in *draw poly* mode, so you can start left-clicking again to draw the next region
  - To exit *draw poly* mode, right-click a second time.
  - If you make a mistake, delete the region and re-draw it; see below
- **Selecting regions:**
  - Selected regions have a red outline, yellow otherwise
  - If only *one* region is selected, clicking the *Draw poly* button will allow you to modify it; you will go back to *draw poly* mode
  - To select a different region, click the *Select* button and choose a different region by clicking on it. Multiple regions can be selected by holding *SHIFT* while clicking.
- **Deleting regions:**
  - Select regions using the select tool (see above)
  - Click the wastebin button to delete them; you will be asked for confirmation
- **Changing the label of a region:**
  - Select regions using the select tool (see above)
  - Use the drop-down (normally reads *UNCLASSIFIED*) within the *Labels* section to change the label
- **If the coloured regions are obscuring parts of the image that you need to see:**
  - Within the *Labels* section, click the *Hide labels* checkbox to hide the labels
  - Uncheck it to show them afterwards
- **When you are done:**
  - When you are satisfied that you have marked out all of the regions of interest and that they are correctly labelled, click the *Finished* checkbox within the *Current image* section. This will mark the image as finished within the system.

### Show the label data

To show the label data in JSON form, execute the code cell below:

In [ ]:
labeller.label_data